In [1]:
import os
from keras.preprocessing import image
import matplotlib.pyplot as plt 
import numpy as np
from keras.utils.np_utils import to_categorical
import random,shutil
from keras.models import Sequential
from keras.layers import Dropout,Conv2D,Flatten,Dense, MaxPooling2D, BatchNormalization
from keras.models import load_model

Using TensorFlow backend.


In [2]:
def generator(dir,
              gen=image.ImageDataGenerator(rescale=1./255),
              shuffle=True,
              batch_size=1,
              target_size=(24,24),
              class_mode='categorical' ):

    return gen.flow_from_directory(dir,
                                   batch_size=batch_size,
                                   shuffle=shuffle,
                                   color_mode='grayscale',
                                   class_mode=class_mode,
                                   target_size=target_size)

BS= 32
TS=(24,24)
train_batch= generator('a2a_data/train',
                       shuffle=True, 
                       batch_size=BS,
                       target_size=TS)
valid_batch= generator('a2a_data/valid',
                       shuffle=True, 
                       batch_size=BS,
                       target_size=TS)
SPE= len(train_batch.classes)//BS
VS = len(valid_batch.classes)//BS
print(SPE,VS)

Found 5422 images belonging to 2 classes.
Found 1160 images belonging to 2 classes.
169 36


In [3]:
model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(24,24,1)),
    MaxPooling2D(pool_size=(1,1)),
    Conv2D(32,(3,3),activation='relu'),
    MaxPooling2D(pool_size=(1,1)), 
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(1,1)),
   
    Dropout(0.25),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')
])






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [5]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit_generator(train_batch, 
                    validation_data=valid_batch,
                    epochs=15,
                    steps_per_epoch=SPE,
                    validation_steps=VS)

model.save('a2a_final.h5', overwrite=True)



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/15
169/169 [==============================] - 20s 116ms/step - loss: 0.5293 - acc: 0.7324 - val_loss: 0.3800 - val_acc: 0.8524
Epoch 2/15
169/169 [==============================] - 2s 11ms/step - loss: 0.3097 - acc: 0.8711 - val_loss: 0.2053 - val_acc: 0.9211
Epoch 3/15
169/169 [==============================] - 2s 10ms/step - loss: 0.2028 - acc: 0.9197 - val_loss: 0.1537 - val_acc: 0.9388
Epoch 4/15
169/169 [==============================] - 2s 10ms/step - loss: 0.1626 - acc: 0.9353 - val_loss: 0.1906 - val_acc: 0.9291
Epoch 5/15
169/169 [==============================] - 2s 11ms/step - loss: 0.1257 - acc: 0.9495 - val_loss: 0.1403 - val_acc: 0.9495
Epoch 6/15
169/169 [==============================] - 2s 11ms/step - loss: 0.1027 - acc: 0.9622 - val_loss: 0.1439 - val_acc: 0.9477
Epoch 7/15
169/169 [==============================] - 2s 11ms/step - loss: 0.0986 - acc: 0.9636 - val_lo

In [6]:
score = model.evaluate_generator(valid_batch, steps=500)
print(score)

[0.1573905555097991, 0.9615040858018387]
